# Orquestador de ventas 360: resumen
Diseño esto para, partiendo de la demanda desagregada: 
- Capturar contratos
- Descargar xlsx de sistemas SAI, CAMUNDA, SAGI, ZOHO y PISP. 
- Consultar exceles internos
- Organizar las órdenes descargadas
- Generar los pickle de las órdenes
Todo lo anterior es la información mínima indispensable para poder generar información de ventas, que será la referencia interna sobre la que desplegaremos el alimentado de reportes y la generación de reglas para dar retro al usuario.

In [2]:
# Importa librerías y chromedriver
import sys
import os
import yaml
import pandas as pd
# Define the root and script directory
folder_root = os.getcwd()  # Get current directory (where Orquestación.ipynb is)
script_folder = os.path.join(folder_root, "Scripts")  # Path to 'Scripts'
# Ensure the script folder is added to sys.path
if script_folder not in sys.path:
    sys.path.append(script_folder)
Libreria_SAI = os.path.join(script_folder, "Libreria_SAI")
if Libreria_SAI not in sys.path:
    sys.path.append(Libreria_SAI)
Libreria_comunes = os.path.join(script_folder, "Libreria_comunes")
if Libreria_comunes not in sys.path:
    sys.path.append(Libreria_comunes)
Libreria_camunda = os.path.join(script_folder, "Libreria_camunda")
if Libreria_camunda not in sys.path:
    sys.path.append(Libreria_camunda)
Libreria_SAGI = os.path.join(script_folder, "Libreria_SAGI")
if Libreria_SAGI not in sys.path:
    sys.path.append(Libreria_SAGI)

Libreria_PREI = os.path.join(script_folder, "Libreria_PREI")
if Libreria_PREI not in sys.path:
    sys.path.append(Libreria_PREI)
from chrome_driver_load import load_chrome

## SAI
Esta consola de jupyter lab es se va a encargar de: 
- Descargar xlsx de sistemas SAI

In [5]:
# Descargar SAI
from SAI_download import SAI_download
# Define your download directory
download_directory = os.path.join(folder_root,'Implementación', 'Descarga de bases')
#print(download_directory)
driver = load_chrome(download_directory)
if driver:
    # Execute the SAI download process
    SAI_download(driver, "EPH -161215-NS9","Epharma2021" )
    # Optionally, close the driver when done
    driver.quit()

An error occurred: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=133.0.6943.53)
Stacktrace:
0   chromedriver                        0x000000010279394c cxxbridge1$str$ptr + 2728228
1   chromedriver                        0x000000010278bfa8 cxxbridge1$str$ptr + 2697088
2   chromedriver                        0x00000001022f1ff0 cxxbridge1$string$len + 93348
3   chromedriver                        0x00000001022daf44 chromedriver + 192324
4   chromedriver                        0x00000001022fe61c cxxbridge1$string$len + 144080
5   chromedriver                        0x0000000102361004 cxxbridge1$string$len + 548024
6   chromedriver                        0x0000000102379a14 cxxbridge1$string$len + 648904
7   chromedriver                        0x000000010232d004 cxxbridge1$string$len + 335032
8   chromedriver                        0x000000010275c1f4 cxxbridge1$str$ptr + 2501068
9   c

## Camunda

In [8]:
from CAMUNDA_login import SAI_camunda

download_directory = os.path.join(folder_root,'Implementación', 'Descarga de bases')

# Initialize the Chrome driver with your custom settings
driver = load_chrome(download_directory)

if driver:
    # Execute the Camunda login and initial automation
    SAI_camunda(driver, "ArmandoJimenez", "N29f6Mwif")
    driver.quit()

Filtra y presiona enter cuando estés listo... 


## SAGI


In [4]:
from SAGI_download import SAGI_download
# Initialize Chrome with your custom settings
download_directory = os.path.join(folder_root,'Implementación', 'Descarga de bases')
driver = load_chrome(download_directory)
if driver:
    # Call the function with your login credentials (username and password)
    SAGI_download(driver, "EPH161215NS9", "Pharmas21", download_directory)
    driver.quit()


Descargamos facturas 
1)2023-2024 o 
2)2024?
 1



*******************************
Navega a la sección elegida: 2023-2024
El script va a seleccionar 50 resultados por página y continuar...



Please navigate to the 2023-2024 section and press Enter to continue. 


Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to output.
Data extracted and added to 

## PISP (PREI)


In [ ]:
from PREI_downloader import PREI_downloader

download_directory = os.path.join(folder_root,'Implementación', 'Descarga de bases')
driver = load_chrome(download_directory)
user = '0000150462'
password = 'EPH161215NS9'
excel_file = os.path.join(folder_root,'Implementación', 'dates.xlsx')
if driver:
    # Call the function with your login credentials (username and password)
    PREI_downloader(driver, user, password, download_directory, excel_file)

    driver.quit()

Processing row 1: Date START = 01/01/2024, Date END = 15/01/2024
Calling input_date for fecha_inicial...
Date passed: 01/01/2024
Attempt 1: Current value (via JS): '__/__/____'
Field cleared successfully on attempt 1
Attempting to input date: 01/01/2024
Date '01/01/2024' entered successfully into the field.
fecha_inicial set successfully.
Calling input_date for fecha_final...
Date passed: 15/01/2024
Attempt 1: Current value (via JS): '__/__/____'
Field cleared successfully on attempt 1
Attempting to input date: 15/01/2024
Date '15/01/2024' entered successfully into the field.
fecha_final set successfully.
Clicking 'buscar' button...
Dates 01/01/2024 to 15/01/2024 were processed
Moving to the next set.
Processing row 2: Date START = 16/01/2024, Date END = 31/01/2024
Calling input_date for fecha_inicial...
Date passed: 16/01/2024
Attempt 1: Current value (via JS): '__/__/____'
Field cleared successfully on attempt 1
Attempting to input date: 16/01/2024
Date '16/01/2024' entered successfu